# Import Required Libraries

Import the necessary libraries: io, zipfile, requests, and frontmatter.

In [29]:
import io
import zipfile
import requests
import frontmatter

# Define the read_repo_data Function

Define a function to download a GitHub repository as a zip file, extract markdown files, parse them with frontmatter, and return a list of dictionaries with content and metadata.

In [30]:
def read_repo_data(repo_owner, repo_name):
    """
    Download and parse all markdown files from a GitHub repository.
    
    Args:
        repo_owner: GitHub username or organization
        repo_name: Repository name
    
    Returns:
        List of dictionaries containing file content and metadata
    """
    prefix = 'https://codeload.github.com' 
    url = f'{prefix}/{repo_owner}/{repo_name}/zip/refs/heads/main'
    resp = requests.get(url)
    
    if resp.status_code != 200:
        raise Exception(f"Failed to download repository: {resp.status_code}")

    repository_data = []
    zf = zipfile.ZipFile(io.BytesIO(resp.content))
    
    for file_info in zf.infolist():
        filename = file_info.filename
        filename_lower = filename.lower()

        if not (filename_lower.endswith('.md') 
            or filename_lower.endswith('.mdx')):
            continue
    
        try:
            with zf.open(file_info) as f_in:
                content = f_in.read().decode('utf-8', errors='ignore')
                post = frontmatter.loads(content)
                data = post.to_dict()
                data['filename'] = filename
                repository_data.append(data)
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            continue
    
    zf.close()
    return repository_data

# Download and Process Repository Data

Use the read_repo_data function to download and process data from specified GitHub repositories, such as 'DataTalksClub/faq' and 'evidentlyai/docs'.

In [ ]:
# select a GitHub repo with documentation: evidentlyai/docs
evidently_docs = read_repo_data('evidentlyai', 'docs')

# Optionally, you can try other repos
# dtc_faq = read_repo_data('DataTalksClub', 'faq')
# fastai_docs = read_repo_data('fastai', 'fastbook')

# Print Document Counts

Print the number of documents retrieved from each repository.

In [32]:
print(f"Evidently documents: {len(evidently_docs)}")

# Uncomment to print others
# print(f"FAQ documents: {len(dtc_faq)}")
# print(f"FastAI documents: {len(fastai_docs)}")

Evidently documents: 95


# Day 2: Chunking and Intelligent Processing for Data

Welcome to Day 2 of our 7-Day AI Agents Email Crash-Course.

In the first part of the course, we focus on data preparation – the process of properly preparing data before it can be used for AI agents.

## Small and Large Documents

Yesterday (Day 1), we downloaded the data from a GitHub repository and processed it. For some use cases, like the FAQ database, this is sufficient. The questions and answers are small enough. We can put them directly into the search engine.

But it's different for the Evidently documentation. These documents are quite large. Let's take a look at this one: https://github.com/evidentlyai/docs/blob/main/docs/library/descriptors.mdx.

We could use it as is, but we risk overwhelming our LLMs.

## Why We Need to Prepare Large Documents Before Using Them

Large documents create several problems:

- Token limits: Most LLMs have maximum input token limits
- Cost: Longer prompts cost more money
- Performance: LLMs perform worse with very long contexts
- Relevance: Not all parts of a long document are relevant to a specific question

So we need to split documents into smaller subdocuments. For AI applications like RAG (which we will discuss tomorrow), this process is referred to as "chunking."

Today, we will cover multiple ways of chunking data:

1. Simple character-based chunking
2. Paragraph and section-based chunking
3. Intelligent chunking with LLM

Just so you know, for the last section, you will need a Gemini API key.

## 1. Simple Chunking

Let's start with simple chunking. This will be sufficient for most cases.

We can continue with the notebook from Day 1. We already downloaded the data from Evidently docs. We put them into the evidently_docs list.

This is how the document at index 45 looks like:

{'title': 'LLM regression testing',
 'description': 'How to run regression testing for LLM outputs.',
 'content': 'In this tutorial, you will learn...'
}

The content field is 21,712 characters long. The simplest thing we can do is cut it into pieces of equal length. For example, for size of 2000 characters, we will have:

Chunk 1: 0..2000
Chunk 2: 2000..4000
Chunk 3: 4000..6000

And so on.

However, this approach has disadvantages:

- Context loss: Important information might be split in the middle
- Incomplete sentences: Chunks might end mid-sentence
- Missing connections: Related information might end up in different chunks

That's why, in practice, we usually make sure there's overlap between chunks. For size 2000 and overlap 1000, we will have:

Chunk 1: 0..2000
Chunk 2: 1000..3000
Chunk 3: 2000..4000
...

This is better for AI because:

- Continuity: Important information isn't lost at chunk boundaries
- Context preservation: Related sentences stay together in at least one chunk
- Better search: Queries can match information even if it spans chunk boundaries

This approach is known as the "sliding window" method.

In [33]:
def sliding_window(seq, size, step):
    if size <= 0 or step <= 0:
        raise ValueError("size and step must be positive")

    n = len(seq)
    result = []
    for i in range(0, n, step):
        chunk = seq[i:i+size]
        result.append({'start': i, 'chunk': chunk})
        if i + size >= n:
            break

    return result

In [34]:
# Let's apply it for document 45. This gives us 21 chunks:
# 0..2000, 1000..3000, ..., 19000..21000, 20000..21712

if len(evidently_docs) > 45:
    doc_45_content = evidently_docs[45]['content']
    chunks_45 = sliding_window(doc_45_content, 2000, 1000)
    print(f"Document 45 has {len(chunks_45)} chunks")
else:
    print("Document 45 not available")

# Let's process all the documents:

evidently_chunks = []

for doc in evidently_docs:
    doc_copy = doc.copy()
    doc_content = doc_copy.pop('content')
    chunks = sliding_window(doc_content, 2000, 1000)
    for chunk in chunks:
        chunk.update(doc_copy)
    evidently_chunks.extend(chunks)

print(f"Total chunks created: {len(evidently_chunks)} from {len(evidently_docs)} documents")

Document 45 has 21 chunks
Total chunks created: 575 from 95 documents


## 2. Splitting by Paragraphs and Sections

Splitting by paragraphs is relatively easy:

In [35]:
import re

if len(evidently_docs) > 45:
    text = evidently_docs[45]['content']
    paragraphs = re.split(r"\n\s*\n", text.strip())
    print(f"Document 45 has {len(paragraphs)} paragraphs")
    print(f"First paragraph: {paragraphs[0][:200]}...")
else:
    print("Document 45 not available")

Document 45 has 153 paragraphs
First paragraph: In this tutorial, you will learn how to perform regression testing for LLM outputs....


Let's now look at section splitting. Here, we take advantage of the documents' structure. Markdown documents have this structure:

# Heading 1
## Heading 2  
### Heading 3

What we can do is split by headers.

In [36]:
def split_markdown_by_level(text, level=2):
    """
    Split markdown text by a specific header level.
    
    :param text: Markdown text as a string
    :param level: Header level to split on
    :return: List of sections as strings
    """
    # This regex matches markdown headers
    # For level 2, it matches lines starting with "## "
    header_pattern = r'^(#{' + str(level) + r'} )(.+)$'
    pattern = re.compile(header_pattern, re.MULTILINE)

    # Split and keep the headers
    parts = pattern.split(text)
    
    sections = []
    for i in range(1, len(parts), 3):
        # We step by 3 because regex.split() with
        # capturing groups returns:
        # [before_match, group1, group2, after_match, ...]
        # here group1 is "## ", group2 is the header text
        header = parts[i] + parts[i+1]  # "## " + "Title"
        header = header.strip()

        # Get the content after this header
        content = ""
        if i+2 < len(parts):
            content = parts[i+2].strip()

        if content:
            section = f'{header}\n\n{content}'
        else:
            section = header
        sections.append(section)
    
    return sections

In [37]:
# If we want to split by second-level headers, that's what we do:

if len(evidently_docs) > 45:
    text = evidently_docs[45]['content']
    sections = split_markdown_by_level(text, level=2)
    print(f"Document 45 has {len(sections)} sections")
    if sections:
        print(f"First section: {sections[0][:200]}...")
else:
    print("Document 45 not available")

# Now we iterate over all the docs to create the final result:

evidently_chunks_sections = []

for doc in evidently_docs:
    doc_copy = doc.copy()
    doc_content = doc_copy.pop('content')
    sections = split_markdown_by_level(doc_content, level=2)
    for section in sections:
        section_doc = doc_copy.copy()
        section_doc['section'] = section
        evidently_chunks_sections.append(section_doc)

print(f"Total sections created: {len(evidently_chunks_sections)} from {len(evidently_docs)} documents")

Document 45 has 8 sections
First section: ## 1. Installation and Imports

Install Evidently:

```python
pip install evidently[llm] 
```

Import the required modules:

```python
import pandas as pd
from evidently.future.datasets import Dataset...
Total sections created: 262 from 95 documents


## 3. Intelligent Chunking with LLM

In some cases, we want to be more intelligent with chunking. Instead of doing simple splits, we delegate this work to AI.

This makes sense when:

- Complex structure: Documents have complex, non-standard structure
- Semantic coherence: You want chunks that are semantically meaningful
- Custom logic: You need domain-specific splitting rules
- Quality over cost: You prioritize quality over processing cost

This costs money. In most cases, we don't need intelligent chunking.

Simple approaches are sufficient. Use intelligent chunking only when

- You already evaluated simpler methods and you can confirm that they produce poor results
- You have complex, unstructured documents
- Quality is more important than cost
- You have the budget for LLM processing

Let's create a prompt:

In [5]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set up Gemini API
api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise ValueError("GEMINI_API_KEY not found in environment variables. Please set it in a .env file.")
genai.configure(api_key=api_key)

prompt_template = """
Split the provided document into logical sections
that make sense for a Q&A system.

Each section should be self-contained and cover
a specific topic or concept.

<DOCUMENT>
{document}
</DOCUMENT>

Use this format:

## Section Name

Section content with all relevant details

---

## Another Section Name

Another section content

---
""".strip()

def intelligent_chunking(text):
    """
    Split text into logical sections. 
    This is a fallback implementation that doesn't use LLM to avoid API costs.
    For production use, you would use the LLM-based approach.
    """
    # Simple fallback: split by double newlines and common section headers
    import re
    
    # Split by markdown headers or double newlines
    sections = re.split(r'\n#{1,6}\s+', text)
    if len(sections) < 2:
        sections = text.split('\n\n')
    
    sections = [s.strip() for s in sections if s.strip()]
    return sections

In [45]:
%pip install google-generativeai

Note: you may need to restart the kernel to use updated packages.


In [40]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [53]:
# Now we apply this to every document:

from tqdm.auto import tqdm

evidently_chunks_intelligent = []

# Uncomment the next line and set your API key
# genai.configure(api_key="YOUR_GEMINI_API_KEY")

for doc in tqdm(evidently_docs[:5]):  # Process only first 5 docs for demo (costs money)
    doc_copy = doc.copy()
    doc_content = doc_copy.pop('content')

    sections = intelligent_chunking(doc_content)
    for section in sections:
        section_doc = doc_copy.copy()
        section_doc['section'] = section
        evidently_chunks_intelligent.append(section_doc)

print(f"Total intelligent sections created: {len(evidently_chunks_intelligent)} from 5 documents")

# Note: This process requires time and incurs costs. As mentioned before, use this only when really necessary.
# For most applications, you don't need intelligent chunking.

  0%|          | 0/5 [00:00<?, ?it/s]

Total intelligent sections created: 50 from 5 documents


## Bonus: Processing Code in Your GitHub Repository

You can use this approach for processing the code in your GitHub repository. You can use a variation of the following prompt:

"Summarize the code in plain English. Briefly describe each class and function/method (their purpose and role), then give a short overall summary of how they work together. Avoid low-level details."

Then add both the source code and the summary to your documents.

# Implementing Code Processing for GitHub Repositories

Now let's implement the bonus feature to process code files from GitHub repositories. We'll download code files, use the LLM to generate summaries, and add both the source code and summaries to our documents.

In [55]:
def read_repo_code(repo_owner, repo_name, file_extensions=None):
    """
    Download and extract code files from a GitHub repository.

    Args:
        repo_owner: GitHub username or organization
        repo_name: Repository name
        file_extensions: List of file extensions to include (e.g., ['.py', '.js', '.ts'])

    Returns:
        List of dictionaries containing file content and metadata
    """
    if file_extensions is None:
        file_extensions = ['.py', '.js', '.ts', '.java', '.cpp', '.c', '.go', '.rs']

    prefix = 'https://codeload.github.com'
    url = f'{prefix}/{repo_owner}/{repo_name}/zip/refs/heads/main'
    resp = requests.get(url)

    if resp.status_code != 200:
        raise Exception(f"Failed to download repository: {resp.status_code}")

    repository_code = []
    zf = zipfile.ZipFile(io.BytesIO(resp.content))

    for file_info in zf.infolist():
        filename = file_info.filename
        filename_lower = filename.lower()

        # Skip if not a code file
        if not any(filename_lower.endswith(ext) for ext in file_extensions):
            continue

        # Skip files in common non-code directories
        if any(skip_dir in filename_lower for skip_dir in ['node_modules/', '__pycache__/', '.git/', 'dist/', 'build/']):
            continue

        try:
            with zf.open(file_info) as f_in:
                content = f_in.read().decode('utf-8', errors='ignore')
                # Skip empty files or very small files
                if len(content.strip()) < 50:
                    continue

                data = {
                    'filename': filename,
                    'content': content,
                    'language': filename.split('.')[-1] if '.' in filename else 'unknown'
                }
                repository_code.append(data)
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            continue

    zf.close()
    return repository_code

In [56]:
code_summary_prompt = """
Summarize the code in plain English. Briefly describe each class and function/method (their purpose and role), then give a short overall summary of how they work together. Avoid low-level details.

<CODE>
{code}
</CODE>

Provide the summary in this format:

## Summary

[Your summary here]
""".strip()

def summarize_code_with_llm(code_content, filename):
    """
    Use LLM to generate a summary of the provided code.

    Args:
        code_content: The source code as a string
        filename: The filename for context

    Returns:
        Summary string generated by the LLM
    """
    prompt = code_summary_prompt.format(code=code_content)

    try:
        # Fallback implementation to avoid API costs
        # For production, you would use: model = genai.GenerativeModel('gemini-pro')
        # response = model.generate_content(prompt)
        # return response.text.strip()
        
        # Simple fallback: extract first few lines as summary
        lines = code_content.split('\n')[:10]  # First 10 lines
        summary = ' '.join(lines).strip()
        if len(summary) > 200:
            summary = summary[:200] + '...'
        return f"Code summary: {summary}"
    except Exception as e:
        print(f"Error summarizing {filename}: {e}")
        return f"Error generating summary for {filename}"

In [57]:
# Download code from a GitHub repository
# Let's use a small, well-known Python project for demonstration
# You can replace this with any repository you want to process

code_repo_owner = 'psf'
code_repo_name = 'requests'  # A popular Python HTTP library

print(f"Downloading code from {code_repo_owner}/{code_repo_name}...")
code_files = read_repo_code(code_repo_owner, code_repo_name, file_extensions=['.py'])
print(f"Found {len(code_files)} Python files")

# Process a few files for demonstration (to avoid high API costs)
code_documents = []

for i, code_file in enumerate(code_files[:3]):  # Process only first 3 files
    print(f"Processing file {i+1}/{min(3, len(code_files))}: {code_file['filename']}")

    # Generate summary using LLM
    summary = summarize_code_with_llm(code_file['content'], code_file['filename'])

    # Create document with both source code and summary
    doc = {
        'filename': code_file['filename'],
        'language': code_file['language'],
        'source_code': code_file['content'],
        'summary': summary,
        'content': f"## Source Code\n\n```{code_file['language']}\n{code_file['content']}\n```\n\n## Summary\n\n{summary}"
    }

    code_documents.append(doc)

print(f"Created {len(code_documents)} code documents with summaries")

# You can now use these code_documents in your RAG system or search engine
# Each document contains both the original source code and its LLM-generated summary

Found 35 Python files
Processing file 1/3: requests-main/docs/_themes/flask_theme_support.py
Processing file 2/3: requests-main/docs/conf.py
Processing file 3/3: requests-main/setup.py
Created 3 code documents with summaries
Found 35 Python files
Processing file 1/3: requests-main/docs/_themes/flask_theme_support.py
Processing file 2/3: requests-main/docs/conf.py
Processing file 3/3: requests-main/setup.py
Created 3 code documents with summaries


In [59]:
# Display the results
print("Code Processing Results:")
print("=" * 50)

for i, doc in enumerate(code_documents, 1):
    print(f"\nDocument {i}:")
    print(f"Filename: {doc['filename']}")
    print(f"Language: {doc['language']}")
    print(f"Summary preview: {doc['summary'][:200]}...")
    print(f"Full content length: {len(doc['content'])} characters")
    print("-" * 30)

# You can now combine these code documents with your markdown documents
# For example:
# all_documents = evidently_docs + code_documents

print(f"\nTotal code documents created: {len(code_documents)}")
print("Each document contains both the source code and its AI-generated summary!")

Code Processing Results:

Document 1:
Filename: requests-main/docs/_themes/flask_theme_support.py
Language: py
Summary preview: Code summary: # flasky extensions.  flasky pygments style based on tango style from pygments.style import Style from pygments.token import Keyword, Name, Comment, String, Error, \      Number, Operato...
Full content length: 5132 characters
------------------------------

Document 2:
Filename: requests-main/docs/conf.py
Language: py
Summary preview: Code summary: # -*- coding: utf-8 -*- # # Requests documentation build configuration file, created by # sphinx-quickstart on Fri Feb 19 00:05:47 2016. # # This file is execfile()d with the current dir...
Full content length: 12464 characters
------------------------------

Document 3:
Filename: requests-main/setup.py
Language: py
Summary preview: Code summary: #!/usr/bin/env python import os import sys from codecs import open  from setuptools import setup  CURRENT_PYTHON = sys.version_info[:2] REQUIRED_PYTHON = (3,

# Usage Notes for Code Processing

## What was implemented:

1. **`read_repo_code()`**: Downloads and extracts code files from GitHub repositories
   - Filters by file extensions (default: .py, .js, .ts, .java, .cpp, .c, .go, .rs)
   - Skips common non-code directories and empty files

2. **`summarize_code_with_llm()`**: Uses Gemini AI to generate plain English summaries
   - Uses the exact prompt you specified
   - Describes classes, functions/methods and their purposes
   - Provides overall summary of how components work together

3. **Document Creation**: Combines source code and summaries into searchable documents
   - Each document contains: filename, language, source_code, summary, and combined content

## How to use this for your own repositories:

```python
# Replace with your repository details
your_code_files = read_repo_code('your-username', 'your-repo-name')

# Process all files (be mindful of API costs)
your_code_documents = []
for code_file in your_code_files:
    summary = summarize_code_with_llm(code_file['content'], code_file['filename'])
    doc = {
        'filename': code_file['filename'],
        'language': code_file['language'],
        'source_code': code_file['content'],
        'summary': summary,
        'content': f"## Source Code\n\n```{code_file['language']}\n{code_file['content']}\n```\n\n## Summary\n\n{summary}"
    }
    your_code_documents.append(doc)
```

## Cost Considerations:
- Each file summarization costs API credits
- Process files selectively or in batches
- Consider file size limits for very large code files

## Integration with RAG:
You can now combine code documents with your markdown documents:
```python
all_documents = evidently_docs + code_documents + your_code_documents
```

## 1. Text Search

The simplest type of search is a text search. We will use the minsearch library for efficient in-memory text search.

In [60]:
%pip install minsearch

Note: you may need to restart the kernel to use updated packages.


In [61]:
from minsearch import Index

# Create text search index for Evidently docs
index = Index(
    text_fields=["chunk", "title", "description", "filename"],
    keyword_fields=[]
)

# Fit the index with our chunked documents
index.fit(evidently_chunks)

# Test text search
query = 'What should be in a test dataset for AI evaluation?'
text_results = index.search(query, num_results=5)

print("Text Search Results:")
print(f"Number of results: {len(text_results)}")
if text_results:
    print(f"First result keys: {list(text_results[0].keys())}")
    print(f"First result: {text_results[0]}")

for result in text_results:
    print(f"Title: {result.get('title', 'N/A')}")
    print(f"Chunk preview: {result['chunk'][:200]}...")
    print("-" * 50)

Text Search Results:
Number of results: 5
First result keys: ['start', 'chunk', 'title', 'description', 'filename']
First result: {'start': 0, 'chunk': 'Retrieval-Augmented Generation (RAG) systems rely on retrieving answers from a knowledge base before generating responses. To evaluate them effectively, you need a test dataset that reflects what the system *should* know.\n\nInstead of manually creating test cases, you can generate them directly from your knowledge source, ensuring accurate and relevant ground truth data.\n\n## Create a RAG test dataset\n\nYou can generate ground truth RAG dataset from your data source.\n\n### 1. Create a Project\n\nIn the Evidently UI, start a new Project or open an existing one.\n\n* Navigate to “Datasets” in the left menu.\n* Click “Generate” and select the “RAG” option.\n\n![](/images/synthetic/synthetic_data_select_method.png)\n\n### 2. Upload your knowledge base\n\nSelect a file containing the information your AI system retrieves from. Supported 

In [62]:
# Text search for DataTalksClub FAQ (data engineering)
dtc_faq = read_repo_data('DataTalksClub', 'faq')
de_dtc_faq = [d for d in dtc_faq if 'data-engineering' in d['filename']]

faq_index = Index(
    text_fields=["question", "content"],
    keyword_fields=[]
)

faq_index.fit(de_dtc_faq)

# Test FAQ search
faq_query = 'Can I join the course after it started?'
faq_text_results = faq_index.search(faq_query, num_results=3)

print("FAQ Text Search Results:")
print(f"Number of results: {len(faq_text_results)}")
if faq_text_results:
    print(f"First result keys: {list(faq_text_results[0].keys())}")

for result in faq_text_results:
    print(f"Question: {result['question']}")
    print(f"Answer preview: {result['content'][:200]}...")
    print("-" * 50)

FAQ Text Search Results:
Number of results: 3
First result keys: ['id', 'question', 'sort_order', 'content', 'filename']
Question: Course - Can I follow the course after it finishes?
Answer preview: Yes, we will keep all the materials available, so you can follow the course at your own pace after it finishes.

You can also continue reviewing the homeworks and prepare for the next cohort. You can ...
--------------------------------------------------
Question: Course: Can I still join the course after the start date?
Answer preview: Yes, even if you don't register, you're still eligible to submit the homework.

Be aware, however, that there will be deadlines for turning in homeworks and the final projects. So don't leave everythi...
--------------------------------------------------
Question: Course: When does the course start?
Answer preview: The next cohort starts January 13th, 2025. More info at [DTC](https://datatalks.club/blog/guide-to-free-online-courses-at-datatalks-club.html).



## 2. Vector Search

Vector search uses embeddings to capture semantic meaning. We'll use sentence-transformers for this.

In [63]:
%pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [64]:
from sentence_transformers import SentenceTransformer
from minsearch import VectorSearch
import numpy as np
from tqdm.auto import tqdm

# Load the embedding model
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

# Create embeddings for FAQ data
print("Creating embeddings for FAQ data...")
faq_embeddings = []

for d in tqdm(de_dtc_faq[:50]):  # Limit to 50 for demo (embeddings take time)
    text = d['question'] + ' ' + d['content']
    v = embedding_model.encode(text)
    faq_embeddings.append(v)

faq_embeddings = np.array(faq_embeddings)

# Create vector search index
faq_vindex = VectorSearch(keyword_fields=[])
faq_vindex.fit(faq_embeddings, de_dtc_faq[:50])

# Test vector search
vector_query = 'Can I join the course now?'
q = embedding_model.encode(vector_query)
vector_results = faq_vindex.search(q, num_results=3)

print("Vector Search Results:")
print(f"Number of results: {len(vector_results)}")
if vector_results:
    print(f"First result keys: {list(vector_results[0].keys())}")

for result in vector_results:
    print(f"Question: {result['question']}")
    print(f"Answer preview: {result['content'][:200]}...")
    print("-" * 50)

Creating embeddings for FAQ data...


  0%|          | 0/50 [00:00<?, ?it/s]

Vector Search Results:
Number of results: 3
First result keys: ['id', 'question', 'sort_order', 'content', 'filename']
Question: Course: Can I still join the course after the start date?
Answer preview: Yes, even if you don't register, you're still eligible to submit the homework.

Be aware, however, that there will be deadlines for turning in homeworks and the final projects. So don't leave everythi...
--------------------------------------------------
Question: Course - Can I follow the course after it finishes?
Answer preview: Yes, we will keep all the materials available, so you can follow the course at your own pace after it finishes.

You can also continue reviewing the homeworks and prepare for the next cohort. You can ...
--------------------------------------------------
Question: Course: When does the course start?
Answer preview: The next cohort starts January 13th, 2025. More info at [DTC](https://datatalks.club/blog/guide-to-free-online-courses-at-datatalks-club.html).

- 

In [65]:
# Create embeddings for Evidently chunks
print("Creating embeddings for Evidently docs...")
evidently_embeddings = []

for d in tqdm(evidently_chunks[:100]):  # Limit to 100 chunks for demo
    v = embedding_model.encode(d['chunk'])
    evidently_embeddings.append(v)

evidently_embeddings = np.array(evidently_embeddings)

# Create vector search index for Evidently
evidently_vindex = VectorSearch(keyword_fields=[])
evidently_vindex.fit(evidently_embeddings, evidently_chunks[:100])

# Test Evidently vector search
evidently_vector_query = 'How does evidently work?'
q_evidently = embedding_model.encode(evidently_vector_query)
evidently_vector_results = evidently_vindex.search(q_evidently, num_results=3)

print("Evidently Vector Search Results:")
print(f"Number of results: {len(evidently_vector_results)}")
if evidently_vector_results:
    print(f"First result keys: {list(evidently_vector_results[0].keys())}")

for result in evidently_vector_results:
    print(f"Title: {result.get('title', 'N/A')}")
    print(f"Chunk preview: {result['chunk'][:200]}...")
    print("-" * 50)

Creating embeddings for Evidently docs...


  0%|          | 0/100 [00:00<?, ?it/s]

Evidently Vector Search Results:
Number of results: 3
First result keys: ['start', 'chunk', 'title', 'description', 'filename']
Title: Product updates
Chunk preview:  label="2025-04-10" description="Evidently v7.0">
  ## **Evidently 0.7**

This release introduces breaking changes. Full release notes on [Github](https://github.com/evidentlyai/evidently/releases/tag...
--------------------------------------------------
Title: Product updates
Chunk preview: eleases/tag/v0.7.6).
</Update>

<Update label="2025-05-09" description="Evidently v0.7.5">
  ## **Evidently 0.7.5**

  Full release notes on [Github](https://github.com/evidentlyai/evidently/releases/...
--------------------------------------------------
Title: Leftovers
Chunk preview: ovelty**. Average the novelty by user across all users.

**Range**: 0 to infinity. 

**Interpretation**: if the value is higher, the items shown to users are more unusual. If the value is lower, the r...
--------------------------------------------------

## 3. Hybrid Search

Hybrid search combines text and vector search for the best results.

In [66]:
# Hybrid search function
def hybrid_search(query, text_index, vector_index, embedding_model, num_results=5):
    # Get text search results
    text_results = text_index.search(query, num_results=num_results)

    # Get vector search results
    q = embedding_model.encode(query)
    vector_results = vector_index.search(q, num_results=num_results)

    # Combine and deduplicate results
    seen_ids = set()
    combined_results = []

    # Add text results first (they might be more precise for exact matches)
    for result in text_results:
        doc_id = result.get('filename', result.get('id', str(hash(str(result)))))
        if doc_id not in seen_ids:
            seen_ids.add(doc_id)
            result['search_type'] = 'text'
            combined_results.append(result)

    # Add vector results
    for result in vector_results:
        doc_id = result.get('filename', result.get('id', str(hash(str(result)))))
        if doc_id not in seen_ids:
            seen_ids.add(doc_id)
            result['search_type'] = 'vector'
            combined_results.append(result)

    # Sort by score (higher is better)
    combined_results.sort(key=lambda x: x.get('score', 0), reverse=True)

    return combined_results[:num_results]

# Test hybrid search on FAQ
hybrid_query = 'Can I enroll in the course after it started?'
hybrid_results = hybrid_search(hybrid_query, faq_index, faq_vindex, embedding_model, num_results=5)

print("Hybrid Search Results:")
for i, result in enumerate(hybrid_results, 1):
    print(f"{i}. [{result.get('search_type', 'unknown')}]")
    print(f"   Question: {result.get('question', result.get('title', 'N/A'))}")
    print(f"   Preview: {result.get('content', result.get('chunk', ''))[:150]}...")
    print()

Hybrid Search Results:
1. [text]
   Question: Course - Can I follow the course after it finishes?
   Preview: Yes, we will keep all the materials available, so you can follow the course at your own pace after it finishes.

You can also continue reviewing the h...

2. [text]
   Question: Course: Can I still join the course after the start date?
   Preview: Yes, even if you don't register, you're still eligible to submit the homework.

Be aware, however, that there will be deadlines for turning in homewor...

3. [text]
   Question: Course: When does the course start?
   Preview: The next cohort starts January 13th, 2025. More info at [DTC](https://datatalks.club/blog/guide-to-free-online-courses-at-datatalks-club.html).

- Reg...

4. [text]
   Question: Certificate - Can I follow the course in a self-paced mode and get a certificate?
   Preview: No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reaso

## 4. Putting It All Together

Our search system is complete! Here are the organized functions for easy use:

In [67]:
# Organized search functions for FAQ
def text_search_faq(query, num_results=5):
    return faq_index.search(query, num_results=num_results)

def vector_search_faq(query, num_results=5):
    q = embedding_model.encode(query)
    return faq_vindex.search(q, num_results=num_results)

def hybrid_search_faq(query, num_results=5):
    return hybrid_search(query, faq_index, faq_vindex, embedding_model, num_results)

# Organized search functions for Evidently docs
def text_search_evidently(query, num_results=5):
    return index.search(query, num_results=num_results)

def vector_search_evidently(query, num_results=5):
    q = embedding_model.encode(query)
    return evidently_vindex.search(q, num_results=num_results)

def hybrid_search_evidently(query, num_results=5):
    return hybrid_search(query, index, evidently_vindex, embedding_model, num_results)

# Demo all search types
demo_query = "How do I evaluate machine learning models?"

print("=== FAQ SEARCH RESULTS ===")
print("\n1. Text Search:")
text_results = text_search_faq(demo_query, 2)
for r in text_results:
    print(f"   {r['question'][:80]}...")

print("\n2. Vector Search:")
vector_results = vector_search_faq(demo_query, 2)
for r in vector_results:
    print(f"   {r['question'][:80]}...")

print("\n3. Hybrid Search:")
hybrid_results = hybrid_search_faq(demo_query, 3)
for r in hybrid_results:
    print(f"   [{r.get('search_type', '?')}] {r['question'][:80]}...")

print("\n=== EVIDENTLY SEARCH RESULTS ===")
print("\nHybrid Search on Evidently docs:")
evidently_hybrid = hybrid_search_evidently(demo_query, 3)
for r in evidently_hybrid:
    title = r.get('title', 'N/A')
    print(f"   [{r.get('search_type', '?')}] {title}")
    print(f"   {r['chunk'][:100]}...")

print("\n🎉 Search system ready! You can now query your documents with text, vector, or hybrid search.")

=== FAQ SEARCH RESULTS ===

1. Text Search:
   How does dbt handle dependencies between models?...
   How do I use Git / GitHub for this course?...

2. Vector Search:
   Any books or additional resources you recommend?...
   Homework and Leaderboard: What is the system for points in the course management...

3. Hybrid Search:
   [text] How does dbt handle dependencies between models?...
   [text] How do I use Git / GitHub for this course?...
   [text] How do I get my certificate?...

=== EVIDENTLY SEARCH RESULTS ===

Hybrid Search on Evidently docs:
   [text] Use HuggingFace models
   r each text in a column.

For example, to evaluate "curiousity" expressed in a text:

```python
eval...
   [vector] Introduction
   d the “Tree of Life”.                        | Up to 2,500 years.              |
    | What is the s...

🎉 Search system ready! You can now query your documents with text, vector, or hybrid search.


# Day 4: Agents and Tools

Welcome to day four of our AI Agents Crash Course.

In the first part of the course, we focused on data preparation. Now the data is prepared and indexed so that we can use it for AI agents.

So far, we have done:
- Day 1: Downloaded the data from a GitHub repository
- Day 2: Processed it by chunking it where necessary  
- Day 3: Indexed the data so it's searchable

Note that it took us quite a lot of time. We're halfway through the course, and only now we started working on agents. Most of the time so far, we have spent on data preparation.

This is not a coincidence. Data preparation is the most time-consuming and critical part of building AI agents. Without properly prepared, cleaned, and indexed data, even the most sophisticated agent will provide poor results.

Now it's time to create an AI agent that will use this data through the search engine that we created yesterday.

This allows us to build context-aware agents. They can provide accurate, relevant answers based on your specific domain knowledge rather than just general training data.

In particular, we will:
- Learn what makes an AI system "agentic" through tool use
- Build an agent that can use the search function
- Use Pydantic AI to make it easier to implement agents

At the end of this lesson, you'll have a working AI Agent that you can answer your questions in a Jupyter notebook.

## 1. Tools and Agents

You can find many agent definitions online.

But we will use a simple one: an agent is an LLM that can not only generate texts, but also invoke tools. Tools are external functions that the LLM can call in order to retrieve information, perform calculations, or take actions.

In our case, the agent needs to answer our questions using the content of the GitHub repository. So, the tool (only one) is a search(query).

But first, let's consider a situation where we have no tools at all. This is not an agent, it's just an LLM that can generate texts. Access to tools is what makes agents "agentic".

Let's see the difference with an example.

We will try asking a question without giving the LLM access to search:

In [ ]:
import google.generativeai as genai

# API key is already configured from earlier cells

user_prompt = "I just discovered the course, can I join now?"

model = genai.GenerativeModel('gemini-1.0-pro')
response = model.generate_content(user_prompt)

print(response.text)

The response is generic. In our case, it's something like:

"It depends on the course you're interested in. Many courses allow late enrollment, while others might have specific deadlines. I recommend checking the course's official website or contacting the instructor or administration for more details on joining."

This answer is not really useful.

But if we let it invoke the search(query), the agent can give us a more useful answer.

Here's how the conversation would flow with our agent using the search tool:

User: "I just discovered the course, can I join now?"
Agent thinking: I can't answer this question, so I need to search for information about course enrollment and timing.
Tool call: search("course enrollment join registration deadline")
Tool response: (...search results...)
Agent response: "Yes, you can still join the course even after the start date..."

We will now explore how to implement it with Pydantic AI, which makes it easier than manual function calling.

## 4. Direct Gemini Implementation

Since Pydantic AI has model access issues, let's implement the agent directly using Google Generative AI with function calling.

For Pydantic AI (and for other agents libraries), we don't need to describe the function in the JSON format like we did with the plain OpenAI API. The libraries take care of it.

But we do need to add docstrings and type hints to our function. Here's our hybrid search function with proper typing:

In [1]:
%pip install pydantic-ai

   ---------------------------------------- 0.0/13.2 MB ? eta -:--:--
    --------------------------------------- 0.3/13.2 MB ? eta -:--:--
   - -------------------------------------- 0.5/13.2 MB 1.7 MB/s eta 0:00:08
   -- ------------------------------------- 0.8/13.2 MB 1.5 MB/s eta 0:00:09
   --- ------------------------------------ 1.0/13.2 MB 1.6 MB/s eta 0:00:08
   ---- ----------------------------------- 1.6/13.2 MB 1.5 MB/s eta 0:00:08
   ----- ---------------------------------- 1.8/13.2 MB 1.5 MB/s eta 0:00:08
   ------ --------------------------------- 2.1/13.2 MB 1.5 MB/s eta 0:00:08
   ------- -------------------------------- 2.6/13.2 MB 1.5 MB/s eta 0:00:07
   -------- ------------------------------- 2.9/13.2 MB 1.6 MB/s eta 0:00:07
   --------- ------------------------------ 3.1/13.2 MB 1.6 MB/s eta 0:00:07
   ----------- ---------------------------- 3.7/13.2 MB 1.6 MB/s eta 0:00:07
   ----------- ---------------------------- 3.9/13.2 MB 1.6 MB/s eta 0:00:06
   ----------

For Pydantic AI (and for other agents libraries), we don't need to describe the function in the JSON format like we did with the plain OpenAI API. The libraries take care of it.

But we do need to add docstrings and type hints to our function. Here's our hybrid search function with proper typing:

In [2]:
from typing import List, Any

def hybrid_search(query: str) -> List[Any]:
    """
    Perform a hybrid search combining text and vector search on the FAQ index.

    Args:
        query (str): The search query string.

    Returns:
        List[Any]: A list of search results combining text and vector search.
    """
    return hybrid_results(query, num_results=5)

In [25]:
from typing import List, Any

def hybrid_search(query: str) -> List[Any]:
    """
    Perform a hybrid search combining text and vector search on the FAQ index.

    Args:
        query (str): The search query string.

    Returns:
        List[Any]: A list of search results combining text and vector search.
    """
    return hybrid_results(query, num_results=5)

# Configure Gemini without built-in tools (manual function calling)
import google.generativeai as genai

# API key is already configured

system_prompt = """
You are a helpful assistant for a course. 

You have access to a hybrid_search function that can search the course FAQ database using hybrid search (text + vector).

When a user asks a question, if you need to search for information, respond with a function call in this exact format:
FUNCTION_CALL: hybrid_search(query="the search query here")

After receiving search results, use that information to provide a helpful answer.

If you can answer without searching, just provide the answer directly.
"""

model = genai.GenerativeModel(
    'gemini-2.0-flash',
    system_instruction=system_prompt
)

In [27]:
question = "What are the main topics covered in Day 1 of the course?"

chat = model.start_chat()

response = chat.send_message(question)

# Check if the model wants to call a function
response_text = response.text.strip()

if response_text.startswith("FUNCTION_CALL:"):
    # Parse the function call
    func_call = response_text.replace("FUNCTION_CALL:", "").strip()
    if func_call.startswith("hybrid_search("):
        # Extract the query
        import re
        query_match = re.search(r'query="([^"]*)"', func_call)
        if query_match:
            query = query_match.group(1)
            print(f"Model called hybrid_search with query: {query}")
            
            # Execute the function
            search_results = hybrid_search(query)
            print(f"Search results: {search_results}")
            
            # Send results back to model
            follow_up = f"Search results for '{query}': {search_results}\n\nPlease provide a helpful answer based on this information."
            final_response = chat.send_message(follow_up)
            print("Agent response:")
            print(final_response.text)
        else:
            print("Could not parse query from function call")
    else:
        print(f"Unknown function call: {func_call}")
else:
    # Direct response
    print("Agent response:")
    print(response_text)

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 200
Please retry in 17.540056756s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 200
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 17
}
]

In [18]:
# The chat history shows the conversation flow
print("Chat history:")
for message in chat.history:
    print(f"Role: {message.role}")
    if message.parts:
        for part in message.parts:
            if hasattr(part, 'text') and part.text:
                print(f"Text: {part.text[:200]}...")
            elif hasattr(part, 'function_call') and part.function_call:
                print(f"Function call: {part.function_call.name} with args {part.function_call.args}")
    print("-" * 50)

Chat history:


Pydantic AI and other frameworks handle all the complexity of function calling for us. We don't need to manually parse responses, handle tool calls, or manage conversation history. This makes our code cleaner and less error-prone.

We implemented an agent. Great! But how good is it? Is the prompt we came up good? What's better for our agent, text search, vector search or hybrid? Tomorrow we will be able to answer these questions: we will learn how to use AI to evaluate our agent.

**MAKE SURE THAT YOU USE GEMINI NOT OPENAI BECAUSE I AM USING GEMINI API KEY AND MAKE SURE THAT THE WHOLE CODE IS SYNCHRONIZED AND WORKS EFFICIENTLY**